In [ ]:
import cv2
from matplotlib import pyplot as plt
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
# pytesseract.pytesseract.tesseract_cmd = r'C:\Users\User\AppData\Local\Tesseract-OCR\tesseract.exe'
import requests
import json
from datetime import datetime

token = "secret_9g"
databaseId = "ec"

headers = {
  "Authorization": "Bearer " + token,
  "Content-Type": "application/json",
  "Notion-Version": "2022-02-22"
}

def readDatabase(databaseId, headers):
  res = requests.request("POST", f"https://api.notion.com/v1/databases/{databaseId}/query", headers=headers)

  with open('./Prayer_Calendar_2.json', 'w', encoding='utf8') as f:
    json.dump(res.json(), f, ensure_ascii=False)

# readDatabase(databaseId, headers)

img = cv2.imread('January_2023.png', 0)
crop = img[1260:img.shape[0]-400, 0:img.shape[1]]
gray = cv2.threshold(crop,200,255,cv2.THRESH_BINARY)[1]
cal = list( filter( None, pytesseract.image_to_string(gray, config=r'--oem 3 --psm 3 -c tessedit_char_blacklist==.‘,()').split("\n") ) )
# plt.imshow(cv2.cvtColor(gray, cv2.COLOR_BGR2RGB))

year = 2023; month = 1

Timings = {"Fajr": [], "Fajr Jammah": [], "Shurooq": [], "Dhuhr": [], "Dhuhr Jammah": [], "Asr": [], 
  "Asr Jammah": [], "Maghrib": [], "Maghrib Jammah": [], "Isha": [], "Isha Jammah": [] }

for dmonth, row in enumerate(cal, start=1):
  for i, (prayer, time) in enumerate(Timings.items(), start=-11):
    hours, min = map( int, row.split()[i].split(':') )
    time.append( datetime(year, month, dmonth, hours, min).isoformat() ) #2023,1,1,6,12 → "2023-01-03T06:12:00"

properties = {}

for prayer, time in Timings.items():
  for i in range(len(cal)):
    properties[ str( datetime(year, month, i+1).strftime("%d/%m/%y") ) ] = { "date": {"start": time[i]} }

  for i in range(31 - len(cal)):
    properties[ str( datetime(year, month, i+1).strftime("%d/%m/%y") ) ] = {"date": None}
    
  properties["Name"] = {  "title": [{ "text": {"content": prayer} }]  }

  newPageData = { "parent": {"database_id": databaseId}, "properties": properties }
  res = requests.request("POST", 'https://api.notion.com/v1/pages', headers=headers, data=json.dumps(newPageData))
  print(res.status_code);# print(res.text)